In [1]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\achra\OneDrive\Desktop\Generative AI\Generative AI Projects\Medical BOT Project\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
PINECONE_API_KEY = '3e06a55f-2c51-4e2d-ac1d-c6cf64430d54'
INECONE_INDEX_KEY = ""

In [8]:
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents = loader.load()
    
    return documents



In [10]:
extracted_data = load_pdf("data/")

In [16]:
extracted_data[0]

Document(page_content='', metadata={'source': 'data\\Medical_book.pdf', 'page': 0})

In [19]:
# Create text chunks 

def text_split(extracted_data):
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [20]:
txt_chunks = text_split(extracted_data)

In [23]:
print(len(txt_chunks))

7020


In [24]:
# download embedding Model

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

In [26]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [27]:
query = embeddings.embed_query('hello hugging face')
print(query)

[-0.023073233664035797, 0.08501016348600388, 0.08825463801622391, 0.049619581550359726, -0.005361923482269049, -0.0665019229054451, 0.027237385511398315, -0.05356225371360779, -0.017672957852482796, -0.062184520065784454, 0.013149291276931763, -0.06592119485139847, -0.02073526382446289, -0.014917000196874142, 0.07317351549863815, -0.01468341238796711, 0.049783024936914444, -0.05022621154785156, -0.060413509607315063, 0.10321291536092758, -0.10170730203390121, 0.02701091393828392, -0.017766345292329788, 0.017803095281124115, -0.01716061308979988, 0.0018529442604631186, 0.06811630725860596, 0.004165003076195717, 0.11930255591869354, -0.021519897505640984, -0.007234236691147089, -0.059751447290182114, 0.04387801140546799, 0.03133412450551987, -0.0166632067412138, 0.08324573934078217, -0.023299379274249077, 0.01056713704019785, -0.05577182397246361, -0.018539737910032272, -0.0763164535164833, -0.02545182779431343, 0.022822227329015732, -0.014686120674014091, 0.06466800719499588, 0.02164237

In [35]:
pc = pinecone.pinecone(api_key=PINECONE_API_KEY)

#index = pc.Index(YOUR_INDEX_NAME)


index_name = "medi-bot"

doc_search = pinecone.from_texts([t.page_content for t in txt_chunks],embeddings,index_name=index_name)

TypeError: 'module' object is not callable